In [65]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/10_1_1.csv')
print(df.head(2))
df_group = df.groupby('소주제')['정답여부'].value_counts().reset_index(drop=False)
df_q1 = pd.DataFrame()
df_q1['소주제'] = df_group.소주제.unique()
df_q1['정답수'] = df_group[df_group['정답여부'] == 1][['소주제','count']].groupby('소주제').sum().reset_index(drop=True)['count']
df_q1['문항수'] = df_group[['소주제','count']].groupby('소주제').sum().reset_index(drop=True)['count']
df_q1['정답률'] = df_q1['정답수'] / df_q1['문항수']
print(df_q1.정답률.value_counts().reset_index(drop=False).sort_values(by='정답률', ascending=False)['정답률'].iloc[2])

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/10_1_2.csv')
print(df.head(2))
df['year'] = df['date'].str[:4]
df['month'] = df['date'].str[5:7]
print(df.groupby(['year', 'month'])['price'].sum().sort_values(ascending=False).iloc[1])
target_year = df.groupby(['year', 'month'])['price'].sum().sort_values(ascending=False).reset_index(drop=False).iloc[3]['year']
target_month =  df.groupby(['year', 'month'])['price'].sum().sort_values(ascending=False).reset_index(drop=False).iloc[3]['month']
target_df = df[(df['year'] == target_year) & (df['month'] == target_month)]
print(target_df.groupby('category')['price'].sum().max())

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/10_1_3.csv')
df['split'] = df['message'].str.split(' ')
df['len'] = df['split'].str.len()
print(df.groupby('label')['len'].mean())
print(round(abs(df.groupby('label')['len'].mean().max() - df.groupby('label')['len'].mean().min()),3))

   학생ID  문제ID 대주제 소주제  정답여부
0  1001     1  과학  동물     0
1  1001     2  사회  정치     1
0.68
         date category  item  price
0  2023-01-03       커피    라떼  46123
1  2023-02-05       커피  카푸치노  77519
1777389
1012500
label
ham     6.232143
spam    6.531915
Name: len, dtype: float64
0.3


In [128]:
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/10_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/10_2_test.csv')
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

train = train[train['gas_totl'] != 0]
y_train = train['gas_totl']
x_train = train.drop('gas_totl', axis=1)
x_test = test.drop('gas_totl', axis=1)

co = pd.concat([x_train, x_test], axis=0)
co = pd.get_dummies(co, columns = ['biz_type'])
num = ['area', 'age', 'num_households']

x_train = co.iloc[:len(x_train),:]
x_test = co.iloc[len(x_train):,:]

x_t, x_va, y_t, y_va = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

model = RandomForestRegressor(
    n_estimators = 1000,
    max_depth = 3,
    min_samples_split = 7,
    min_samples_leaf = 5,
    random_state = 42
)
model.fit(x_t, y_t)
pred = model.predict(x_va)
print('RMSE:', root_mean_squared_error(y_va, pred))
print(y_train.max(), y_train.min())
result = pd.DataFrame()
result['pred'] = model.predict(x_test)
print(result.isnull().sum())
print(result.head(3))
#result.to_csv('result.csv', index=False)

RMSE: 69.17015709924534
1820 56
pred    0
dtype: int64
          pred
0   540.962793
1  1016.728126
2   357.305106


In [125]:
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/10_3_1.csv')
print(df.head(2))
import statsmodels.formula.api as smf
model = smf.logit('attrition ~ age + income + C(overtime)', data=df).fit()
print(round(model.params[(model.pvalues < 0.05) & (model.params.index != 'Intercept')],3)) 
import numpy as np
print(round(np.exp(model.params['age']), 3))
test_data = pd.DataFrame()
test_data['age'] = [20]
test_data['income'] = [3000]
test_data['overtime'] = [2]
pred = model.predict(test_data)
print(round(pred.item(),3))

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/10_3_2.csv')
print(df.head(2))
model = smf.ols('price ~ area + height + C(wall)', data=df).fit()
print(round(model.params[(model.pvalues < 0.05) & (model.params.index != 'Intercept')].sum(),3))
model2 = smf.ols('price ~ area + height', data=df).fit()
print(round(model2.rsquared ,3))
test_data2 = pd.DataFrame()
test_data2['area'] = [100]
test_data2['height'] = [10]
test_data2['wall'] = [1]
pred = model2.predict(test_data2)
print(round(pred.item(),3))

   attrition  age  income  overtime
0          0   51    9270         0
1          0   23    9603         1
         Current function value: 0.048045
         Iterations: 35
income   -0.005
dtype: float64
0.894
0.48
        price       area     height  wall
0  317.312359  89.934283  10.715575     0
1  273.821798  77.234714  11.121569     0
10.289
0.859
329.036


C:\Users\icedo\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
